# Connexion au cluster

La connexion est possible uniquement depuis les PCs des salles de TP  

```bash
$ ssh pnavaro@cluster-infomaths.insa-rennes.fr
The authenticity of host 'cluster-infomaths.insa-rennes.fr (193.52.94.112)' can't be established.
ECDSA key fingerprint is SHA256:wEtf2Oc8PKFls/D9oDQF4vXcs9y2ku0i+At4KiVJ04Q.
Are you sure you want to continue connecting (yes/no)? yes
Warning: Permanently added 'cluster-infomaths.insa-rennes.fr,193.52.94.112' (ECDSA) to the list of known hosts.
pnavaro@cluster-infomaths.insa-rennes.fr's password:
Linux cim-tete 4.9.0-4-amd64 #1 SMP Debian 4.9.65-3+deb9u1 (2017-12-23) x86_64

The programs included with the Debian GNU/Linux system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Debian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent
permitted by applicable law.
Last login: Tue Oct 16 09:02:56 2018 from 10.4.1.36
pnavaro@cim-tete:~$ 
```

# Commandes de base

- **sbatch**  : soumission d'un job dans une partition.
- **scancel** : Suppression d'un job.
- **squeue**  : interrogation des jobs.
- **sinfo**   : interrogation des files d'attente.
- **srun**    : execution immediate d'une commande.
- **salloc**  : batch interactif, obtention d'un shell, permettant d'enchaîner plusieurs commandes sur les mêmes ressources.  
- **sprio**   : priorités relatives entre les jobs en attente  

Une documentation des commandes de base est disponible ici : [http://slurm.schedmd.com/man_index.html](http://slurm.schedmd.com/man_index.html) 

# Utilisation du Cluster

## Ressources disponibles

- **Noeuds :** 4 (cim-n01 / cim-n02 / cim-n03 / cim-n04)  
- **CPU par noeud :** 24  
- **Socket par noeud :** 1  
- **Coeur par socket :** 12  
- **Threads par coeur :** 2  
- **RAM par noeud :** 64Go  
- **Nom du cluster :** cluster-infomath  
- **Nom de la partition :** queue 

## Lancement d'un job

Pour lancer un job, il faut créer un script dans lequel il faut demander des ressources puis appeler son programme (voir les exemples plus loin).  

Ce script est ensuite soumi au gestionnaire de file d'attente avec la commande **sbatch**. Par exemple :  

```
$ sbatch mon_script.sh
```

On obtient alors un numéro de job, qui peut être manipulé avec les commandes `scancel` ou `squeue`.  

Le script peut être écrit dans le langage de son choix (bash, tcsh, python, perl...). Il peut être exécuté directement, sans être appelé par sbatch, et dans ce cas, les directives d'allocations de ressources seront ignorées, et il s'exécutera dans le shell local.  

`
## Demande de ressources

Vos besoins en terme de ressources sont décrits dans l'en-tête d'un fichier via des directives Slurm. Par exemple :  

```slurm
#SBATCH -N 4  
#SBATCH --tasks-per-node 5
```

demandera une allocation de 4 noeuds à 5 coeurs chacun (par défaut, 1 tâche demande 1 coeur).

D'autres critères peuvent être spécifié via ces directives, comme la taille mémoire souhaitée ou la durée pendant laquelle les ressources seront attribuées.  

**Plus vous serez parcimonieux dans la demande d'allocation, plus vous aurez de chance de voir rapidement votre job passer de l'état en attente à l'état en exécution.**  

Par exemple, s'il est possible d'estimer précisément la durée nécessaire à une exécution, il peut être profitable de réduire au minimum la durée demandée pour la réservation. Ainsi une exécution se faisant en 3h30 pourra se faire au sein d'un job demandant 4h00 (marge de 30mn par précaution), avec la directive  

```
#SBATCH -t 04:00:00  
```

Attention toutefois à prendre une marge suffisante, car au-delà du temps demandé, l'exécution est stoppée automatiquement par Slurm.

Attention : toutes les partitions sont configurées avec une limite de temps d'exécution par défaut, qui s'applique à tout job ne précisant pas combien de temps doit lui être alloué. Pour connaître cette limite, utiliser la commande suivante :  

```
sinfo -o "%10P %.11L %.11l"
```

Remarque : tous les arguments de la directive #SBATCH peuvent également être utilisés en arguments des commandes srun, salloc et sbatch. Voir les exemples plus loin.

## Suivre l'état d'un job

Il est possible d'obtenir le détail de l'état d'un job, qu'il soit en attente ou en exécution, avec les commandes scontrol ou squeue. La commande sview permet également en partie d'obtenir ces informations.  

**Liste des jobs en cours :**  

```
squeue -u
```

**Détail de l'état d'un job :**

```
show job
```

**Mon job est en attente. Quand va-t-il démarrer ?**  

Pour les jobs en attente, Slurm calcul périodiquement un temps probable de démarrage.  

2 possibilités pour l'obtenir:  

```
scontrol show job <jobid>| grep StartTime=</jobid>
```

```
squeue -o "%S" -j
```

## Variables d'environnement

Dans certains cas, votre programme a besoin de connaître plus précisément les ressources que Slurm a mis à sa disposition. Pour cela, Slurm fourni un certain nombre de variables d'environnement, qui sont utilisable dans le script qui appelle le programme.  

- SLURM_NPROCS : nombre de coeurs alloués  
- SLURM_NNODES : nombre de noeuds alloués  
- SLURM_JOB_ID : job id  
- SLURM_JOB_NODELIST : liste des noeuds alloués, sous une forme synthétique. Pour obtenir une liste détaillée, on peut utiliser la commande "scontrol show hostname"

## Priorités

Quand plusieurs jobs sont en même temps en attente dans une file, Slurm calcule une priorité entre ces jobs. Le job ayant la priorité la plus élevée sera le prochain à passer en exécution.  

La priorité des jobs peut être vue avec la commande `sprio -l`.  

La priorité dépend de plusieurs facteurs :  
- le temps d'attente déjà écoulé (AGE)  
- la taille du job en nombre de coeurs : les gros jobs sont favorisés (JOBSIZE)  
- la consommation en heures cpu sur le passé récent : plus la consommation a été faible, plus la priorité augmentera (FAIRSHARE).  

# Exemples

## Exemple de script Slurm simple

Ce script demande 36 coeurs répartis sur au minimum sur 3 noeuds et au maximum sur 6 noeuds :  

```bash
#! /bin/bash  
#SBATCH -p public  
#SBATCH -A lol # voir chapitre "Autorisations / Account" ci-dessus  
#SBATCH -n 36 # 36 coeurs  
#SBATCH -N 3-6 # au minimum 3 noeuds, au maximum 6  
#SBATCH -t 12:00:00 # Le job sera tué au bout de 12h  
#SBATCH --mem=1024 # Quantité mémoire demandée par noeud en Mo (unité obligatoire)  
#SBATCH --mail-type=END # Réception d'un mail à la fin du job  
#SBATCH --mail-user=mon@adresse  

mpirun mon_code
```

## Exemple de script Slurm 

- Spécification de la répartition des tâches par noeud

Si vous avez besoin d’indiquer exactement la répartition des tâches sur les noeuds, vous pouvez utiliser tout ou partie des paramètres `--tasks-per-node`, `--distribution` et éventuellement `--cpu_bind` pour avoir un état de la répartition.  

Vous pouvez par exemple demander 4 coeurs par noeud sur 3 noeuds, avec une répartition cyclique des tâches sur les noeuds, cad la 1ère tâche sur le noeud 1, la 2nd sur le 2nd noeud, la 3ème sur le 3ème noeud, puis la 4ème sur le 1er noeud et ainsi de suite :  

```bash
#!/bin/bash  
#SBATCH -p grant  
#SBATCH -A grant18 # voir chapitre "Autorisations / Account" ci-dessus  
#SBATCH -N 3 # 3 noeuds  
#SBATCH --tasks-per-node=4 # 4 tâches par noeud (par défaut 1 tâche = 1 coeur)  
# La répartition par défaut des tâches est de type block, cad toutes les tâches du début  
# sur le 1er noeud, les tâches suivantes sur le 2nd, etc.. :  
#SBATCH --distribution=cyclic  
#SBATCH --cpu_bind=verbose # produira un état donnant la correspondance tâche/noeud/coeur  

# Environnement par défaut : contient les compilateurs Intel 11  
source /home/configfiles/bashrc.default  

# On remplace par le compilateur Intel 13  
module delete compilers/intel11  
module load compilers/intel13  

# Ajoutons OpenMPI  
module load mpi/openmpi-1.6.i13  

# Ajoutons les lib math. Intel  
module load libs/mkl13  

# Enfin, lançons simplement le job  
# le mpirun de openmpi est interfacé nativement avec Slurm  
# Ainsi, nul besoin de préciser le nombre de processeurs et la liste  
# des machines  
cd ${HOME}/somewhere_i_want_to_work  
mpirun my_mpi_executable  
```

## Exemple de demande de ressources

Les commandes srun, salloc, sbatch peuvent prendre les même paramètres que ceux de la directive #SBATCH.  

Par exemple, la commande suivante demandera 4 coeurs et lancera 1 fois par coeur la commande hostname  

```
$ srun -n 4 hostname  
cim-n01  
cim-n01  
cim-n01  
cim-n01
```

Autre exemple en précisant la répartition :  

```
$ srun -n 4 --tasks-per-node=2 hostname  
cim-n01  
cim-n01  
cim-n02  
cim-n02  
```

Exemple avec la commande salloc, qui permet de travailler en interactif dans une réservation. Demandons 32 coeurs :  

```
$ salloc -n 32  
salloc: Granted job allocation 472  
$  
```

On pourrait avoir l'impression qu'il ne s'est rien passé, cependant :  

- en pratique, les ressources ont été réservées  
dans l'environnement du shell crée, Slurm a positionné des variables qui indiqueront aux jobs parallèles (MPI, Slurm) les ressources à utiliser Par exemple, la commande srun hostname (lancée dans le shell précédemment obtenu), produira la sortie suivante :  

```$ srun hostname  
cim-n01  
cim-n01  
... (24 fois)  
cim-n02  
cim-n02  
... (8 fois)  
```

## Exemple de lancement d'un job MPI

OpenMPI est couplé nativement avec Slurm. Par défaut, il lancera autant de processus que de coeurs disponibles.  

Il est possible d'éviter que les processus MPI se déplacent d'un coeur à l'autre par l'option --bind-to-core de mpirun  

Pour les applications mpi hybrides (X processus lançant chacun Y threads), il faut :  
- réserver X*Y coeurs avec slurm, en indiquant qu'on prend tous les coeurs d'un noeud (--tasks-per-node=16)  
- lancer mpirun -npernode 2 -bind-to-socket -npersocket 1 mon_code  
- mpirun obtient la liste des machines de slurm  

## Exemple de lancement d'un job OpenMP

Par défaut les threads des jobs OpenMP seront regroupés de manière compacte sur les coeurs d'un même socket.  

Par exemple, pour soumettre un job OpenMP sur 16 coeurs et dans un script, les commandes seraient les suivantes :  

```bash
#! /bin/bash  

#SBATCH -n 16 --tasks-per-node=16  

export OMP_NUM_THREADS=16  

./mon_code_omp
```

## Exemple de lancement d'un job hybride MPI/OpenMP

Pour exécuter par exemple 8 tâches MPI lançant chacune 4 thread OpenMP, il faut un total de 32 coeurs  

Tout d'abord, il faut indiquer dans le script le nombre de threads OpenMP que doit lancer chaque process MPI : 

```bash
export OMP_NUM_THREADS=4
```

Ensuite, chaque tâche MPI aura besoin de 4 coeurs obligatoirement sur le même noeud (pour que chaque thread OpenMP ait son propre coeur). Il faut donc faire une des demandes de ressources suivantes à Slurm :  
Pour obtenir 8 noeuds différents avec 4 coeurs chacun :  

```bash
#SBATCH --tasks-per-node=4  
#SBATCH -N 8  
```

ou alors 4 noeuds différents avec 8 coeurs (version adaptée aux noeuds 8 coeurs de la partition pri2008)  

```bash
#SBATCH --tasks-per-node=8  
#SBATCH -N 4  
```

ou encore 2 noeuds différents avec 16 coeurs (version adaptée aux noeuds Equip@meso 16 coeurs)  

```bash
#SBATCH --tasks-per-node=16  
#SBATCH -N 2  
```

Enfin, lors du lancement du programme par mpirun, il faut également indiquer le nombre de tâches qu'il faut lancer par noeud, à l'aide de l'option **-npernode** ainsi que le nombre de tâches par socket avec **-npersocket**. Cette dernière option est très importante, car les thread OpenMP lancés par un processus MPI sont toujours localisés sur le même socket que celui sur lequel tourne le process MPI.  

Par exemple, pour le cas 2 ci-dessus, sur 4 noeuds, on lance 2 tâches MPI par noeud, 1 par socket :  

```bash
mpirun -npernode 2 -npersocket 1 mon_programme
```

ou pour le cas 3 ci-dessus, sur 2 noeuds, on lance 4 tâches MPI par noeud, 2 par socket :  

```
mpirun -npernode 4 -npersocket 2 mon_programme
```



**INSA Rennes / Département INFO © 2017**